In [1]:
#upgrade transformers by installing both upgrade and simple transformers
!pip install --upgrade transformers
!pip install simpletransformers

Requirement already up-to-date: transformers in /usr/local/lib/python3.7/dist-packages (4.3.2)


In [2]:
#import packages needed 
import nltk
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
#open and read and text file
text = open("/content/train_text.txt", 'r')
tweet_text = text.readlines()

In [4]:
tweet_text

['"QT @user In the original draft of the 7th book, Remus Lupin survived the Battle of Hogwarts. #HappyBirthdayRemusLupin" \n',
 '"Ben Smith / Smith (concussion) remains out of the lineup Thursday, Curtis #NHL #SJ" \n',
 'Sorry bout the stream last night I crashed out but will be on tonight for sure. Then back to Minecraft in pc tomorrow night. \n',
 "Chase Headley's RBI double in the 8th inning off David Price snapped a Yankees streak of 33 consecutive scoreless innings against Blue Jays \n",
 '@user Alciato: Bee will invest 150 million in January, another 200 in the Summer and plans to bring Messi by 2017" \n',
 "@user LIT MY MUM 'Kerry the louboutins I wonder how many Willam owns!!! Look Kerry Warner Wednesday!' \n",
 '"\\"""" SOUL TRAIN\\"""" OCT 27 HALLOWEEN SPECIAL ft T.dot FINEST rocking the mic...CRAZY CACTUS NIGHT CLUB ..ADV ticket $10 wt out costume $15..." \n',
 'So disappointed in wwe summerslam! I want to see john cena wins his 16th title \n',
 '"This is the last Sunday w/o

In [5]:
#open and read and labels text file
text = open("/content/train_labels.txt", 'r')
text = text.readlines()

In [6]:
text

['2\n',
 '1\n',
 '1\n',
 '1\n',
 '2\n',
 '2\n',
 '2\n',
 '0\n',
 '2\n',
 '1\n',
 '1\n',
 '1\n',
 '2\n',
 '0\n',
 '1\n',
 '1\n',
 '2\n',
 '2\n',
 '0\n',
 '1\n',
 '1\n',
 '2\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '2\n',
 '2\n',
 '1\n',
 '1\n',
 '2\n',
 '1\n',
 '2\n',
 '2\n',
 '1\n',
 '1\n',
 '2\n',
 '1\n',
 '1\n',
 '2\n',
 '2\n',
 '0\n',
 '2\n',
 '1\n',
 '1\n',
 '2\n',
 '2\n',
 '1\n',
 '1\n',
 '0\n',
 '2\n',
 '1\n',
 '2\n',
 '1\n',
 '2\n',
 '1\n',
 '1\n',
 '1\n',
 '0\n',
 '0\n',
 '2\n',
 '2\n',
 '2\n',
 '1\n',
 '1\n',
 '0\n',
 '0\n',
 '1\n',
 '1\n',
 '1\n',
 '2\n',
 '1\n',
 '0\n',
 '1\n',
 '1\n',
 '2\n',
 '0\n',
 '2\n',
 '1\n',
 '2\n',
 '0\n',
 '2\n',
 '2\n',
 '2\n',
 '1\n',
 '0\n',
 '1\n',
 '2\n',
 '2\n',
 '2\n',
 '1\n',
 '2\n',
 '1\n',
 '2\n',
 '0\n',
 '2\n',
 '1\n',
 '0\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '2\n',
 '2\n',
 '2\n',
 '0\n',
 '2\n',
 '0\n',
 '2\n',
 '1\n',
 '2\n',
 '1\n',
 '1\n',
 '1\n',
 '2\n',
 '1\n',
 '2\n',
 '0\n',
 '2\n',
 '2\n',


In [7]:
#remove \n and change strings into integers
def preprocess_label(x):
    new_text = []
    for t in x:
        t = t.replace("\n", "")
        t = int(t)
        new_text.append(t)
    return (new_text)

tweet_label = preprocess_label(text) 

In [8]:
tweet_label

[2,
 1,
 1,
 1,
 2,
 2,
 2,
 0,
 2,
 1,
 1,
 1,
 2,
 0,
 1,
 1,
 2,
 2,
 0,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 0,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 0,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 0,
 0,
 2,
 2,
 2,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 2,
 0,
 2,
 1,
 2,
 0,
 2,
 2,
 2,
 1,
 0,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 0,
 2,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 0,
 2,
 2,
 2,
 2,
 1,
 1,
 0,
 2,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 2,
 2,
 0,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 0,
 1,
 0,
 1,
 2,
 2,
 1,
 2,
 0,
 0,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 0,
 2,
 2,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 2,
 1,
 0,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 0,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 0,
 2,
 1,
 1,
 2,
 0,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 2,
 0,
 1,
 2,
 1,
 2,
 2,
 1,
 0,
 2,
 1,
 2,
 0,
 2,
 2,
 1,


In [9]:
# Create a pandas dataframe and create two columns with both datasets
df_tweeteval = pd.DataFrame({'tweet':tweet_text, 'label_no':tweet_label})
df_tweeteval

,tweet,label_no
0,"""QT @user In the original draft of the 7th boo...",2
1,"""Ben Smith / Smith (concussion) remains out of...",1
2,Sorry bout the stream last night I crashed out...,1
3,Chase Headley's RBI double in the 8th inning o...,1
4,@user Alciato: Bee will invest 150 million in ...,2
...,...,...
45610,"@user \""""So amazing to have the beautiful Lady...",2
45611,"9 September has arrived, which means Apple's n...",2
45612,Leeds 1-1 Sheff Wed. Giuseppe Bellusci securin...,2
45613,@user no I'm in hilton head till the 8th lol g...,1


In [10]:
model_args = ClassificationArgs(num_train_epochs=1, overwrite_output_dir=True, manual_seed=42)
model = ClassificationModel(model_type='roberta', model_name='roberta-base', use_cuda=True, num_labels=3, args=model_args)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [11]:
model.train_model(df_tweeteval)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:446: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/5702 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(5702, 0.686465304531745)

In [12]:
text = open("/content/test_text.txt", 'r')
tweet_text_2 = text.readlines()

In [13]:
tweet_text_2

["@user @user what do these '1/2 naked pics' have to do with anything? They're not even like that. \n",
 'OH: “I had a blue penis while I was this” [playing with Google Earth VR] \n',
 "@user @user That's coming, but I think the victims are going to be Medicaid recipients. \n",
 'I think I may be finally in with the in crowd #mannequinchallenge  #grads2014 @user \n',
 '@user Wow,first Hugo Chavez and now Fidel Castro. Danny Glover, Michael Moore, Oliver Stone, and Sean Penn are running out of heroes. \n',
 'Savchenko now Saakashvili took drug test live on Ukraine TV. To prove they are not drug-fueled loonies? \n',
 'How many more days until opening day? 😩 \n',
 "Twitter's #ThankYouObama Shows Heartfelt Gratitude To POTUS \n",
 'All CSG and Fracking all around Australia is to cease and Mining Entities held Accountable! - Sign the Petition! \n',
 '@user @user @user @user @user @user take away illegals and dead people and Trump wins popular vote too. \n',
 'An interesting security vulnera

In [14]:
text = open("/content/test_labels.txt", 'r')
text = text.readlines()

In [15]:
def preprocess_label(x):
    new_text = []
    for t in x:
        t = t.replace("\n", "")
        t = int(t)
        new_text.append(t)
    return (new_text)

tweet_label_2 = preprocess_label(text) 

In [16]:
tweet_label_2

[1,
 1,
 1,
 2,
 0,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 2,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 0,
 2,
 1,
 0,
 1,
 0,
 1,
 1,
 2,
 1,
 0,
 1,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 0,
 1,
 1,
 2,
 1,
 1,
 1,
 0,
 1,
 2,
 2,
 0,
 0,
 1,
 1,
 1,
 2,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 2,
 2,
 2,
 0,
 1,
 1,
 1,
 1,
 2,
 0,
 1,
 2,
 1,
 2,
 2,
 0,
 0,
 2,
 1,
 1,
 0,
 2,
 0,
 1,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 2,
 0,
 1,
 1,
 1,
 2,
 0,
 1,
 1,
 0,
 0,
 2,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 2,
 0,
 0,
 1,
 1,
 0,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 1,
 2,
 0,
 2,
 0,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 0,
 0,
 0,
 2,
 2,
 1,
 0,
 1,
 2,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 0,
 2,
 1,
 1,
 0,
 2,
 1,
 2,
 0,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 1,
 2,
 2,


In [17]:
df_tweeteval_test = pd.DataFrame({'tweet':tweet_text_2, 'label_no':tweet_label_2})
df_tweeteval_test

,tweet,label_no
0,@user @user what do these '1/2 naked pics' hav...,1
1,OH: “I had a blue penis while I was this” [pla...,1
2,"@user @user That's coming, but I think the vic...",1
3,I think I may be finally in with the in crowd ...,2
4,"@user Wow,first Hugo Chavez and now Fidel Cast...",0
...,...,...
12279,Sentinel Editorial: FBI’s Comey ‘had no one of...,1
12280,perfect pussy clips #vanessa hudgens zac efron...,1
12281,#latestnews 4 #newmexico #politics + #nativeam...,1
12282,Trying to have a conversation with my dad abou...,0


In [18]:
result, model_outputs, wrong_predictions = model.eval_model(df_tweeteval_test)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1026: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Running Evaluation:   0%|          | 0/1536 [00:00<?, ?it/s]

In [19]:
result

{'eval_loss': 0.645877082036653, 'mcc': 0.5511128952905638}

In [20]:
predictions = []
for x in model_outputs:
  predictions.append(np.argmax(x))
print("f1 score:", f1_score(df_tweeteval_test["label_no"], predictions, average='macro',))


f1 score: 0.7100603570824107
